# Stigma against Opioid Use Disorder varies by Personal Use status

```{margin} 
**To follow the full analysis, click through the hidden analysis code below**
```

In [1]:
%matplotlib inline

In [2]:
# import packages
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from utils import *
from samplics.estimation import TaylorEstimator
pd.set_option('mode.chained_assignment', None)

### Data cleaning/pre-processing

In [3]:
# inputs
STATE_ABBREVIATIONS = "state_abbrev_mappings.json"
DATAPATH = "P:/3652/Common/HEAL/y3-task-c-collaborative-projects/jcoin-stigma/analyses/data/protocol2/"
DATA_FILE = DATAPATH+"3645_JCOIN_HEAL Initiative 2021_NORC_Jan2022_1.sav"
STRATA_FILE = DATAPATH+"VSTRAT_VPSU_Survey_2039_HEAL_MAIN_21_05_14.csv"

In [4]:
# import data and metadata (data dictionaries)
df, meta = pyreadstat.read_sav(DATA_FILE,apply_value_formats=True)


In [5]:

# lower-case column names 
df.columns = df.columns.str.lower()

In [7]:
vars_of_interest = ["caseid",'p_over','weight1','weight2','stigma_scale_score','expanded_10item_stigma','state','age4','racethnicity','educ5','personaluse_ever','familyuse_ever','personalcrimjust_ever','familycrimjust_ever']
categorical_vars = ['p_over','state','age4','racethnicity','educ5',
    'personaluse_ever','familyuse_ever',
    'personalcrimjust_ever','familycrimjust_ever']



In [8]:
# to enable more granular analysis of the stigma scale score(s) - e.g. parsing impact of current versus past OUD on stigma - bring in the individual ss questions

ss_6_past = ['ss_a_historywork','ss_b_historymarry']
ss_6_current = ['ss_c_currentwork','ss_d_currentmarry','ss_e_dangerous','ss_f_trust']

ss_6_full = ss_6_past + ss_6_current

ss_10_past = ['ss_historysteal', 'ss_historyhighrisk']
ss_10_current = ['ss_currentsteal', 'ss_currenthighrisk']

ss_10_full = ss_6_full + ss_10_past + ss_10_current

ss_past = ss_6_past + ss_10_past
ss_current = ss_6_current + ss_10_current



In [10]:
# to enable parsing of stigma by political affiliation, views on race/ethnicity, and experience of racial/ethnic discrimination bring in variables assessing those items

# political = ['pid1','pida','pidb','pidi','partyid7','partyid5']
political = ['partyid7','partyid5']

race = ['race_whiteadvantage','race_rich']

# race_whiteadvanctage: [White people in the U.S. have certain advantages because of the color of their skin.] Do you disagree or agree with the following statements?
    # agree corresponds with recognition of white advantage

# race_rich: [Everyone who works hard, no matter what race they are, has an equal chance to become rich.] Do you disagree or agree with the following statements?
    # agree corresponds with lack of recognition of white advantage

discrimination_experience = ['times_atschool', 'times_hired', 'times_atwork', 'times_housing', 'times_medcare', 'times_restaurant', 'times_credit', 'times_street', 'times_police']

# possible approach: 
    # add count of dicrimination experiences (times) across categories
    # higher numbers mean more discrimination experience



In [16]:

additional_vars_of_interest = ss_10_full + political + race + discrimination_experience

In [17]:
# narrow down the dataset to only a few interesting (and relatively clean, straightforward variables) - check for missingness and impute to fill in missing
sub_df_1 = df[vars_of_interest]
sub_df_2 = df[additional_vars_of_interest]

In [18]:
sub_df_2

,ss_a_historywork,ss_b_historymarry,ss_c_currentwork,ss_d_currentmarry,ss_e_dangerous,ss_f_trust,ss_historysteal,ss_historyhighrisk,ss_currentsteal,ss_currenthighrisk,...,race_rich,times_atschool,times_hired,times_atwork,times_housing,times_medcare,times_restaurant,times_credit,times_street,times_police
0,Neither disagree nor agree,Somewhat disagree,Somewhat disagree,Strongly disagree,Strongly agree,Somewhat agree,Somewhat disagree,Neither disagree nor agree,Strongly agree,Somewhat agree,...,Neither disagree nor agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Somewhat disagree,Neither disagree nor agree,Strongly disagree,Somewhat disagree,Somewhat agree,Neither disagree nor agree,Somewhat disagree,Strongly agree,Somewhat agree,Strongly agree,...,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Neither disagree nor agree,Somewhat disagree,Somewhat disagree,Neither disagree nor agree,Somewhat agree,...,Somewhat disagree,NaN,Once,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Somewhat agree,Neither disagree nor agree,Strongly disagree,Strongly disagree,Somewhat agree,Somewhat agree,Neither disagree nor agree,Neither disagree nor agree,Strongly agree,Strongly agree,...,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Somewhat agree,Neither disagree nor agree,Neither disagree nor agree,Strongly disagree,Neither disagree nor agree,Strongly disagree,Strongly disagree,Strongly disagree,Somewhat agree,Somewhat agree,...,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6510,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Strongly agree,Strongly agree,Neither disagree nor agree,Somewhat agree,Strongly agree,Strongly agree,...,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6511,Somewhat disagree,Somewhat agree,Strongly disagree,Somewhat disagree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,Strongly agree,Strongly agree,...,Somewhat agree,NaN,Two or three times,Two or three times,NaN,NaN,NaN,NaN,NaN,NaN
6512,Neither disagree nor agree,Strongly disagree,Neither disagree nor agree,Strongly disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,...,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6513,Somewhat agree,Strongly agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Neither disagree nor agree,Neither disagree nor agree,...,Somewhat disagree,NaN,NaN,Two or three times,Four or more times,NaN,Two or three times,NaN,Four or more times,NaN


In [19]:
# get all var types
print("var info: ")
print(sub_df_2.info())

# all new vars are categorical

var info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6515 entries, 0 to 6514
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   ss_a_historywork     6471 non-null   category
 1   ss_b_historymarry    6459 non-null   category
 2   ss_c_currentwork     6476 non-null   category
 3   ss_d_currentmarry    6468 non-null   category
 4   ss_e_dangerous       6459 non-null   category
 5   ss_f_trust           6486 non-null   category
 6   ss_historysteal      6480 non-null   category
 7   ss_historyhighrisk   6485 non-null   category
 8   ss_currentsteal      6478 non-null   category
 9   ss_currenthighrisk   6481 non-null   category
 10  pid1                 6472 non-null   category
 11  pida                 2464 non-null   category
 12  pidb                 1540 non-null   category
 13  pidi                 2453 non-null   category
 14  partyid7             6515 non-null   category
 15  partyid5  

In [20]:
# check for missing I

# check if missing values
print("missing values: ")
print(sub_df_2.isnull().sum())

# check if missing values
print("missing values: ")
print(sub_df_2.isna().sum())

# every var has at least some missing, except for partyid7

missing values: 
ss_a_historywork         44
ss_b_historymarry        56
ss_c_currentwork         39
ss_d_currentmarry        47
ss_e_dangerous           56
ss_f_trust               29
ss_historysteal          35
ss_historyhighrisk       30
ss_currentsteal          37
ss_currenthighrisk       34
pid1                     43
pida                   4051
pidb                   4975
pidi                   4062
partyid7                  0
partyid5                 32
race_whiteadvantage      44
race_rich                34
times_atschool         5232
times_hired            5272
times_atwork           5141
times_housing          5943
times_medcare          6028
times_restaurant       5317
times_credit           5924
times_street           5005
times_police           5786
dtype: int64
missing values: 
ss_a_historywork         44
ss_b_historymarry        56
ss_c_currentwork         39
ss_d_currentmarry        47
ss_e_dangerous           56
ss_f_trust               29
ss_historysteal          35
s

In [24]:
print("ss single questions, categories: ")
print(sub_df_2.ss_a_historywork.value_counts(dropna=False))

print("party id 7 composite question, categories: ")
print(sub_df_2.partyid7.value_counts(dropna=False))

print("party id 5 composite question, categories: ")
print(sub_df_2.partyid5.value_counts(dropna=False))

print("discrimination times single questions, categories: ")
print(sub_df_2.times_atschool.value_counts(dropna=False))

ss single questions, categories: 
Somewhat agree                2832
Neither disagree nor agree    1481
Strongly agree                1420
Somewhat disagree              517
Strongly disagree              221
NaN                             44
Name: ss_a_historywork, dtype: int64
party id 7 composite question, categories: 
Strong Democrat                1529
Don't Lean/Independent/None     984
Not so strong Democrat          937
Strong Republican               842
Lean Democrat                   796
Not so strong Republican        706
Lean Republican                 689
Unknown                          32
Name: partyid7, dtype: int64
party id 5 composite question, categories: 
Democrat                       2466
Republican                     1548
Don't Lean/Independent/None     984
Lean Democrat                   796
Lean Republican                 689
NaN                              32
Name: partyid5, dtype: int64
discrimination times single questions, categories: 
NaN              

In [ ]:
# clean up some of the categoricals to be consistently coded
sub_df_1.familycrimjust_ever.replace({0:"No",1:"Yes"},inplace=True)
sub_df_1.familyuse_ever.replace({" No":"No"},inplace=True)
sub_df_1.personalcrimjust_ever.replace({"Yes, ever arrested or incarcerated":"Yes", "No, never arrested or incarcerated":"No"},inplace=True)


In [ ]:
# check for missing 

print(sub_df_1.isnull().sum())


In [ ]:

# impute any missing - confirm missing eliminated

# # impute missing stigma scale score vals with median, impute missing personaluse_ever with mode, "No"

# replace missing values of personaluse_ever with mode value of 'No'
sub_df_1.personaluse_ever.fillna('No',inplace=True)
#print(sub_df_1.isnull().sum())

sub_df_1.familyuse_ever.fillna('No',inplace=True)
#print(sub_df_1.isnull().sum())

sub_df_1.personalcrimjust_ever.fillna('No',inplace=True)
#print(sub_df_1.isnull().sum())

sub_df_1.familycrimjust_ever.fillna('No',inplace=True)
#print(sub_df_1.isnull().sum())


# impute missing stigma scale score values as the median score 
#sub_df_1['stigma_scale_score'].fillna(sub_df_1.groupby('time-point')['stigma_scale_score'].transform('median'),inplace=True)
sub_df_1['stigma_scale_score'].fillna(sub_df_1['stigma_scale_score'].median(),inplace=True)
sub_df_1['expanded_10item_stigma'].fillna(sub_df_1['expanded_10item_stigma'].median(),inplace=True)

print(sub_df_1.isnull().sum())

In [ ]:
# add df column with state 2 letter code
# https://pythonfix.com/code/us-states-abbrev.py/
# state name to two letter code dictionary
us_state_to_abbrev = json.loads(Path(STATE_ABBREVIATIONS).read_text())
state_cd = sub_df_1.state.replace(us_state_to_abbrev)
sub_df_1.insert(6,"state_cd",state_cd,True)

In [ ]:
# Add jcoin information
jcoin_json = json.loads(Path("jcoin_states.json").read_text())

jcoin_df = (pd.DataFrame(jcoin_json)
    .assign(hub_types=lambda df:df["hub"]+"("+df["type"]+")")
    .groupby('states')
    # make a list of the name and type of hub/study and how many hubs are in that state
    .agg({"hub_types":lambda s:",".join(s),"hub":"count"})
    .reset_index()
    .rename(
        columns={"states":"state_cd",
        "hub":"jcoin_hub_count",
        "hub_types":"jcoin_hub_types"})
)

jcoin_df["jcoin_flag"] = 1

In [ ]:
jcoin_df.head()

In [ ]:
sub_df_1 = sub_df_1.merge(jcoin_df,on="state_cd",how="left")
sub_df_1["jcoin_hub_types"].fillna("not JCOIN",inplace=True)
sub_df_1["jcoin_hub_count"].fillna(0,inplace=True)
sub_df_1["jcoin_flag"].fillna(0,inplace=True)
sub_df_1["is_jcoin_hub"] = np.where(sub_df_1["jcoin_hub_types"]=="not JCOIN","No","Yes")
sub_df_1.head()

In [ ]:
# o	all: n/weighted n in genpop
# o	all:n/weighted n in as oversample
# o	all:n/weighted n in as oversample + gen pop(in oversampled state)
# o	per state: n/weighted n in as oversample
# o	missingness; imputation procedures

In [ ]:
pop_counts_by_sampletypexstate = (
    sub_df_1
    .convert_dtypes()
    .assign(jcoin_hub_count=lambda df: df.jcoin_hub_count.astype(str))
    .groupby(['state_cd','p_over'])
    ["stigma_scale_score"]
    .count()
    .unstack(['p_over'])
)
pop_counts_by_sampletypexstate["total"] = pop_counts_by_sampletypexstate.sum(axis=1)

In [ ]:
# merge jcoin info
pop_counts_by_sampletypexstate = pop_counts_by_sampletypexstate\
    .merge(jcoin_df,on='state_cd',how='left')\
    .sort_values("total",ascending=False)\
    .assign(
        jcoin_hub_count=lambda df:df.jcoin_hub_count.fillna(0).astype(int),
        jcoin_flag=lambda df:df.jcoin_flag.fillna(0).astype(int),
        jcoin_hub_types=lambda df:(
            np.where(df.jcoin_hub_types.isna() & df["AS oversample"]>0,"non JCOIN comparison",
                np.where(df.jcoin_hub_types.isna(),"non JCOIN gen pop",df.jcoin_hub_types)
        )
        
        ))

In [ ]:
print("N for Oversample, General and Total By State")
pop_counts_by_sampletypexstate.reset_index()

pop_counts_by_sampletypexstate.to_csv("state_counts.csv")


In [ ]:
print("N All")
pop_counts_by_sampletypexstate.sum().to_frame().T

### National estimates

### Add and correct strata and PSUs necessary for variance estimation 

In [ ]:
strata_df = pd.read_csv(STRATA_FILE)
strata_df.columns = strata_df.columns.str.lower()

# collapse strata containing only 1 PSU
onepsu = (
    strata_df[["vstrat32","vpsu32"]]
    .drop_duplicates()
    .groupby("vstrat32")
    .count()
    .squeeze()
    .loc[lambda s:s==1]
    .index
)
strata_df["vstrat32_corrected"] = strata_df["vstrat32"].where(cond=lambda s:~s.isin(onepsu),other=-1)
# rename PSUs so no duplicates
strata_df["vpsu32_corrected"] = strata_df.groupby(["vstrat32_corrected","vpsu32"]).ngroup()

In [ ]:
# join strata into dataset
sub_df_1 = sub_df_1.set_index("caseid").join(strata_df.set_index('caseid'))

### Data overview (counts and missing values)

In [ ]:
# o	table/bar: 
# 	point estimate + bootstrap CI; based on gen pop and weight1 
# o	table/bar: 
# 	point estimate + bootstrap CI; based on as oversample + gen pop and weight 2
# o	histogram: 
# 	distribution; based on as oversample + gen pop and weight 2
# 	distribution by personaluse_ever, familyuse_ever, personalcrimjust_ever, familycrimjust_ever; based on as oversample + gen pop and weight 2


In [ ]:
sample_estimator = TaylorEstimator("mean")
sample_estimator.estimate(
    y=sub_df_1["stigma_scale_score"],
    samp_weight=sub_df_1["weight2"],
    stratum=sub_df_1["vstrat32_corrected"],
    psu=sub_df_1["vpsu32_corrected"],
)
sample_estimator.to_dataframe()

In [ ]:
state_estimator = TaylorEstimator("mean")
state_estimator.estimate(
    y=sub_df_1["stigma_scale_score"],
    samp_weight=sub_df_1["weight2"],
    stratum=sub_df_1["vstrat32_corrected"],
    psu=sub_df_1["vpsu32_corrected"],
    domain=sub_df_1["state_cd"]
)
state_mean_estimates = state_estimator.to_dataframe().rename(columns={"_domain":"state_cd","_estimate":"mean"})


In [ ]:
state_mean_estimates

In [ ]:
# quick map of state level estimates

fig = px.choropleth(state_mean_estimates,
    locations="state_cd",
    locationmode="USA-states",
    scope="usa",
    color="mean",
    color_continuous_scale="Viridis_r")

fig.show()


In [ ]:
# merge jcoin info
state_mean_estimates = state_mean_estimates\
    .merge(jcoin_df,on='state_cd',how='left')\
    .sort_values("mean",ascending=False)\
    .assign(
        jcoin_hub_count=lambda df:df.jcoin_hub_count.fillna(0).astype(int),
        jcoin_flag=lambda df:df.jcoin_flag.fillna(0).astype(int))

state_mean_estimates["jcoin_color"] = state_mean_estimates.jcoin_flag
state_mean_estimates.jcoin_color.replace({0:"blue",1:"red"},inplace=True)


state_mean_estimates.head()

```{margin} 
**To go to the data/study page on the HEAL Data Platform, follow this link:** my link
```

```{margin} 
**To go to an interactive analytic cloud workspace with the analysis code and data loaded, follow this link:** my link
```

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Sodales ut eu sem integer vitae justo eget. Pellentesque dignissim enim sit amet venenatis urna cursus. Sed faucibus turpis in eu mi bibendum. Scelerisque felis imperdiet proin fermentum leo. Volutpat est velit egestas dui id ornare arcu. Quis lectus nulla at volutpat diam ut venenatis tellus. Tellus pellentesque eu tincidunt tortor aliquam nulla facilisi cras. Pellentesque adipiscing commodo elit at imperdiet dui. 
<br>

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Sodales ut eu sem integer vitae justo eget. Pellentesque dignissim enim sit amet venenatis urna cursus. Sed faucibus turpis in eu mi bibendum. Scelerisque felis imperdiet proin fermentum leo. Volutpat est velit egestas dui id ornare arcu. Quis lectus nulla at volutpat diam ut venenatis tellus. Tellus pellentesque eu tincidunt tortor aliquam nulla facilisi cras. Pellentesque adipiscing commodo elit at imperdiet dui. 
<br><br>
In hac habitasse platea dictumst quisque sagittis purus. Libero volutpat sed cras ornare. Sit amet consectetur adipiscing elit pellentesque habitant morbi tristique senectus. Auctor augue mauris augue neque gravida in fermentum et. Amet mattis vulputate enim nulla aliquet porttitor. Proin sed libero enim sed faucibus turpis in eu. Morbi tristique senectus et netus et malesuada. Feugiat sed lectus vestibulum mattis ullamcorper.

**Data Citation** 
<br>
Harold Pollack, Johnathon Schneider, Bruce Taylor. JCOIN 026: Brief Stigma Survey. Chicago, IL: Center for Translational Data Science HEAL Data Platform (distributor) via Center for Translational Data Science JCOIN Data Commons (repository & distributor), 2022-04-08. (HEAL Data Platform branded doi goes here)
<br>
**Brief Article Citation** 
<br>
What format should this be? 